<a href="https://colab.research.google.com/github/ykim879/soccer.webscrapping/blob/master/Soccer_webscrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import requests
from bs4 import BeautifulSoup
import re
baseURL = 'https://www.soccerbase.com'

In [71]:
request = requests.get("https://www.soccerbase.com/players/home.sd")
soup = BeautifulSoup(request.content, "lxml")
testing = soup.find('div', id = 'players-menu')#attrs = {'class': 'soccerColumn soccerColumnLast'})
for div in testing.find_all('div', class_ = 'soccerColumn'):
  t = re.search('Region', div.text)
  
  print(t)
  if (t == None):
    print('Heck no')
    #print(div)
#for li in testing.find_all('ol'):
#  print(li)

<_sre.SRE_Match object; span=(0, 6), match='Region'>
None
Heck no
None
Heck no
None
Heck no


In [46]:
#Decorator hopefully
import time
def timer(func):
  def timeIt(*args, **kwargs):
    start = time.time()
    func(*args, **kwargs)
    print(f'{time.time() - start:0.2f}', 'seconds to run.')
  return timeIt

In [37]:
#From player page for a team
def getTeamData(team, url, q):
  names = [team]
  request = requests.get(url)
  soup = BeautifulSoup(request.content, "lxml")
  test = soup.find('ul', attrs={'class':'infoList infoListBullet infoListLeftPadding infoListRightPadding'})
  for elem in test.find_all('b'):
    name = elem.find('a')
    names.append(name.text)
  #return names
  q.put(names)

In [72]:
#From competition page of teams
import threading
import queue
@timer
def readTeamPage():
  request = requests.get("https://www.soccerbase.com/tournaments/tournament.sd?comp_id=1")
  soup = BeautifulSoup(request.content, "lxml")
  teamNames = soup.find('table', class_ = 'table')
  teams = {}
  threads = []
  q = queue.Queue()

  for num, url in enumerate(teamNames.find_all('a'), 1):
    #print(num, url['href'])
    #t = threading.Thread(target = getTeamData, args = (url.text, baseURL + url['href'], q))
    #threads.append(t)
    #t.start()
    #print(url.text)
    getTeamData(url.text, baseURL+url['href'], q)
    #print(getTeamData(baseURL + url['href']))
  #print(num)
  for t in threads:
    t.join()
  while not q.empty():
    teamData = q.get()
    teams[teamData[0]] = teamData[1:]

  print(teams)
  '''
  for team, players in teams.items():
    print("TEAM:", team, end = '\nPLAYERS: ')
    print(*players, sep=', ')
    print()
'''

In [73]:
readTeamPage()

{'Arsenal': ['Bernd Leno', 'Emiliano Martinez', 'Matt Macey', 'Deyan Iliev', 'Danny Ballard', 'Jordi Osei-Tutu', 'Hector Bellerin', 'Kieran Tierney', 'William Saliba', 'Sokratis', 'Rob Holding', 'Cedric Soares', 'Shkodran Mustafi', 'Calum Chambers', 'Pablo Mari', 'David Luiz', 'Saed Kolasinac', 'Tolaji Bola', 'Ben Sheaf', 'Bukayo Saka', 'Mesut Ozil', 'Lucas Torreira', 'Ainsley Maitland-Niles', 'Mohamed Elneny', 'Joe Willock', 'Matteo Guendouzi', 'Emile Smith Rowe', 'Granit Xhaka', 'Robbie Burton', 'James Olayinka', 'Alexandre Lacazette', 'Willian', 'Pierre-Emerick Aubameyang', 'Nicolas Pepe', 'Reiss Nelson', 'Eddie Nketiah', 'Gabriel Martinelli', 'Folarin Balogun', 'Tyreece John-Jules'], 'Aston Villa': ['Tom Heaton', 'Jed Steer', 'Orjan Haskjold Nyland', 'Lovre Kalinic', 'Neil Taylor', 'Ezri Konsa', 'James Bree', 'Matt Targett', 'Bjorn Engels', 'Frederic Guilbert', 'Ahmed Elmohamady', 'Kortney Hause', 'Dominic Revan', 'Tyrone Mings', 'Myles Sohna', 'Ben Chrisene', 'Douglas Luiz', 'John